In [2]:
!uv pip install grpcio grpcio-tools

Resolved 5 packages in 824ms                                         
Prepared 2 packages in 1.06s                                             
Installed 5 packages in 17ms                                
 + grpcio==1.76.0
 + grpcio-tools==1.76.0
 + protobuf==6.33.1
 + setuptools==80.9.0
 + typing-extensions==4.15.0


Generate Python code from the proto file

In [7]:
!uv run python -m grpc_tools.protoc -I./proto --python_out=. --grpc_python_out=. ./proto/topictrend.proto


In [1]:
import grpc
from grpc import StatusCode
import logging
import topictrend_pb2
import topictrend_pb2_grpc
from datetime import datetime, timedelta

channel_options = [
    ('grpc.keepalive_time_ms', 30000),
    ('grpc.keepalive_timeout_ms', 5000),
    ('grpc.keepalive_permit_without_calls', True),
    ('grpc.http2.max_pings_without_data', 0),
    ('grpc.http2.min_time_between_pings_ms', 10000),
]

class TopicTrendClient:
    def __init__(self, host='localhost', port=50051):
        self.channel = grpc.insecure_channel(f'{host}:{port}', options=channel_options)
        self.stub = topictrend_pb2_grpc.TopicTrendServiceStub(self.channel)

    def close(self):
        self.channel.close()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
        
    def _handle_grpc_error(self, e: grpc.RpcError):
        """Handle common gRPC errors"""
        error_mapping = {
            StatusCode.NOT_FOUND: "Resource not found",
            StatusCode.INVALID_ARGUMENT: "Invalid request parameters",
            StatusCode.INTERNAL: "Internal server error",
            StatusCode.UNAVAILABLE: "Service unavailable",
            StatusCode.DEADLINE_EXCEEDED: "Request timeout",
        }

        error_msg = error_mapping.get(e.code(), f"Unknown error: {e.details()}")
        self.logger.error(f"gRPC Error [{e.code()}]: {error_msg}")
        raise Exception(f"gRPC Error: {error_msg}")


Find category views for a given time interval

In [2]:
with TopicTrendClient() as client:
    request = topictrend_pb2.CategoryViewsRequest(
            wiki="enwiki",
            category_qid=1458083,
            start_date="2025-11-01",
            end_date="2025-11-30",
            depth=2
    )
    try:
        response = client.stub.GetCategoryViews(request)
        print(f"Retrieved {len(response.views)} daily view records")
        for view in response.views[:5]:  # Show first 5 records
            print(f"Date: {view.date}, Views: {view.views}")
    except grpc.RpcError as e:
        self._handle_grpc_error(e)


Retrieved 30 daily view records
Date: 2025-11-01, Views: 0
Date: 2025-11-02, Views: 854192
Date: 2025-11-03, Views: 981547
Date: 2025-11-04, Views: 986264
Date: 2025-11-05, Views: 972172


In [18]:
"""Get raw article pageviews data"""
with TopicTrendClient() as client:
    request = topictrend_pb2.ArticleViewsRequest(
        wiki="enwiki",
        article_qid=789012,
        start_date="2025-01-01",
        end_date="2025-01-31"
    )

    try:
        response = client.stub.GetArticleViews(request)
        print(f"Retrieved {len(response.views)} daily view records")
        for view in response.views[:5]:  # Show first 5 records
            print(f"Date: {view.date}, Views: {view.views}")
    except grpc.RpcError as e:
        print(f"RPC failed: {e.code()}, {e.details()}")


Retrieved 31 daily view records
Date: 2025-01-01, Views: 0
Date: 2025-01-02, Views: 0
Date: 2025-01-03, Views: 0
Date: 2025-01-04, Views: 0
Date: 2025-01-05, Views: 0


In [19]:

"""Convert QIDs to titles"""
with TopicTrendClient() as client:
    request = topictrend_pb2.TitlesByQidsRequest(
        wiki="enwiki",
        qids=[123456, 789012, 345678]
    )

    try:
        response = client.stub.GetTitlesByQids(request)
        print("QID to Title mapping:")
        for qid, title in response.titles.items():
            print(f"Q{qid}: {title}")
    except grpc.RpcError as e:
        print(f"RPC failed: {e.code()}, {e.details()}")



RPC failed: StatusCode.INTERNAL, Database error: pool timed out while waiting for an open connection


In [22]:

"""Get child categories of a category"""
with TopicTrendClient() as client:
    request = topictrend_pb2.ChildCategoriesRequest(
        wiki="enwiki",
        category_qid=123456
    )

    try:
        response = client.stub.GetChildCategories(request)
        print(f"Found {len(response.category_qids)} child categories:")
        for qid in response.category_qids[:10]:  # Show first 10
            print(f"Q{qid}")
    except grpc.RpcError as e:
        print(f"RPC failed: {e.code()}, {e.details()}")



Found 0 child categories:


In [23]:

"""Get top categories with raw data (no titles)"""
with TopicTrendClient() as client:
    request = topictrend_pb2.TopCategoriesRawRequest(
        wiki="enwiki",
        start_date="2025-01-01",
        end_date="2025-01-31",
        limit=10
    )
    
    try:
        response = client.stub.GetTopCategoriesRaw(request)
        print(f"Top {len(response.categories)} categories:")
        for cat in response.categories:
            print(f"Category Q{cat.category_qid}: {cat.total_views} views")
            print(f"  Top articles: {len(cat.top_articles)}")
            for art in cat.top_articles[:3]:  # Show first 3 articles
                print(f"    Q{art.article_qid}: {art.total_views} views")
    except grpc.RpcError as e:
        print(f"RPC failed: {e.code()}, {e.details()}")



Top 0 categories:


In [26]:
"""Check if a category exists"""
with TopicTrendClient() as client:
    request = topictrend_pb2.ValidateCategoryRequest(
        wiki="enwiki",
        category_qid=123456
    )
    
    try:
        response = client.stub.ValidateCategoryExists(request)
        print(f"Category Q123456 exists: {response.exists}")
    except grpc.RpcError as e:
        print(f"RPC failed: {e.code()}, {e.details()}")


Category Q123456 exists: False


In [ ]:

"""Get category pageviews with titles """
with TopicTrendClient() as client:
    request = topictrend_pb2.CategoryTrendRequest(
        wiki="enwiki",
        category="Machine_learning",
        depth=0,
        start_date="2025-01-01",
        end_date="2025-01-31"
    )

    try:
        response = client.stub.GetCategoryPageviews(request)
        print(f"Category: {response.title} (Q{response.qid})")
        print(f"Views data: {len(response.views)} days")
        print(f"Top articles: {len(response.top_articles)}")

        for article in response.top_articles[:5]:
            print(f"  {article.title}: {article.views} views")
    except grpc.RpcError as e:
        print(f"RPC failed: {e.code()}, {e.details()}")


In [ ]:

"""Get top categories with titles (legacy endpoint)"""
with TopicTrendClient() as client:
    request = topictrend_pb2.TopCategoriesRequest(
        wiki="enwiki",
        start_date="2025-01-01",
        end_date="2025-01-31",
        top_n=10
    )

    try:
        response = client.stub.GetTopCategories(request)
        print(f"Top {len(response.categories)} categories:")

        for cat in response.categories:
            print(f"\n{cat.title} (Q{cat.qid}): {cat.views} views")
            print(f"  Top articles:")
            for art in cat.top_articles[:3]:
                print(f"    {art.title}: {art.views} views")
    except grpc.RpcError as e:
            print(f"RPC failed: {e.code()}, {e.details()}")


In [ ]:

# Async client example
import asyncio
import grpc.aio

"""Example using async gRPC client"""
async with grpc.aio.insecure_channel('localhost:50051') as channel:
    stub = topictrend_pb2_grpc.TopicTrendServiceStub(channel)

    request = topictrend_pb2.TitlesByQidsRequest(
        wiki="enwiki",
        qids=[123456, 789012]
    )

    try:
        response = await stub.GetTitlesByQids(request)
        print("Async result:")
        for qid, title in response.titles.items():
            print(f"Q{qid}: {title}")
    except grpc.RpcError as e:
        print(f"Async RPC failed: {e.code()}, {e.details()}")


In [ ]:

# Batch processing example

"""Example of efficient batch processing"""
with TopicTrendClient() as client:
    # Get a list of category QIDs first
    categories_request = topictrend_pb2.TopCategoriesRawRequest(
        wiki="enwiki",
        start_date="2025-01-01",
        end_date="2025-01-31",
        limit=5
    )

    categories_response = client.stub.GetTopCategoriesRaw(categories_request)
    qids = [cat.category_qid for cat in categories_response.categories]

    # Batch convert QIDs to titles
    titles_request = topictrend_pb2.TitlesByQidsRequest(
        wiki="enwiki",
        qids=qids
    )

    titles_response = client.stub.GetTitlesByQids(titles_request)

    # Now we have both raw data and titles
    print("Categories with titles:")
    for cat in categories_response.categories:
        title = titles_response.titles.get(cat.category_qid, f"Q{cat.category_qid}")
        print(f"{title}: {cat.total_views} views")